## Transfer Learning on Binary Classification
Lets try transfer learning on binary classification images with pizza_steak

In [ ]:
# download pizza_steak file to local folder
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/pizza_steak.zip

import zipfile

# unzip the downloaded file
zip_ref = zipfile.ZipFile("pizza_steak.zip")
zip_ref.extractall()
zip_ref.close()

--2021-06-26 05:54:55--  https://storage.googleapis.com/ztm_tf_course/food_vision/pizza_steak.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.193.128, 173.194.195.128, 173.194.196.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.193.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 109579078 (105M) [application/zip]
Saving to: ‘pizza_steak.zip’

pizza_steak.zip     100%[===================>] 104.50M   120MB/s    in 0.9s    

2021-06-26 05:54:56 (120 MB/s) - ‘pizza_steak.zip’ saved [109579078/109579078]



In [ ]:
import os

# walkthru pizza_steak directory
for dirpath, dirnames, filenames in os.walk("pizza_steak"):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

There are 2 directories and 1 images in 'pizza_steak'.
There are 2 directories and 1 images in 'pizza_steak/train'.
There are 0 directories and 750 images in 'pizza_steak/train/steak'.
There are 0 directories and 750 images in 'pizza_steak/train/pizza'.
There are 2 directories and 1 images in 'pizza_steak/test'.
There are 0 directories and 250 images in 'pizza_steak/test/steak'.
There are 0 directories and 250 images in 'pizza_steak/test/pizza'.


In [ ]:
# setup data inputs
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SHAPE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 5

# directories
train_dir = "pizza_steak/train/"
test_dir = "pizza_steak/test/"

# preprocess
train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

# import data
print("Training Images")
train_data = train_datagen.flow_from_directory(train_dir,
                                               target_size=IMAGE_SHAPE,
                                               batch_size=BATCH_SIZE,
                                               class_mode="binary")
print("Test Images")
test_data = test_datagen.flow_from_directory(test_dir,
                                             target_size=IMAGE_SHAPE,
                                             batch_size=BATCH_SIZE,
                                             class_mode="binary")


Training Images
Found 1500 images belonging to 2 classes.
Test Images
Found 500 images belonging to 2 classes.


## Setup Callbacks

In [ ]:
# create tensorboard callback
import datetime
import tensorflow as tf

def create_tensorboard_callback(dir_name, experiment_name):
  log_dir = dir_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
  print(f"Saving Tensorboard log files to : {log_dir}")
  return tensorboard_callback
  

## Create model using tensorflow hub

In [ ]:
efficientnet_url = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"
vgnnet_url = "https://tfhub.dev/google/imagenet/inception_v3/feature_vector/5"
mobilenet_url = "https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/4"

In [ ]:
# import dependencies
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers

import matplotlib.pyplot as plt


In [ ]:
# create model from url

def create_model(model_url, num_classes=2):
  """
  Using TensorFlow Hub create Sequential Model

  Args :
  - Model URL : TensorFlow Hub feature extraction URL
  - num_classes : number of output neurons in output_layer

  returns :
  uncompiled sequential model
  """

  # download pre-trained model and save it as keras layer
  feature_extractor_layer = hub.KerasLayer(model_url,
                                           trainable=False,
                                           name="feature_extractor_layer",
                                           input_shape=IMAGE_SHAPE+(3,))
  
  # create model
  model = tf.keras.Sequential()
  model.add(feature_extractor_layer)
  model.add(layers.Dense(num_classes, activation="sigmoid", name="output_layer"))

  return model

## Mobilenet Model

In [ ]:
# create mobilenet model
mobilenet_model = create_model(mobilenet_url,
                            num_classes=train_data.num_classes)


In [ ]:
# compile mobilenet model
mobilenet_model.compile(loss="binary_crossentropy",
                     optimizer=tf.keras.optimizers.Adam(),
                     metrics=["accuracy"])


In [ ]:
# mobilenet model summary
mobilenet_model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
feature_extractor_layer (Ker (None, 1280)              2257984   
_________________________________________________________________
output_layer (Dense)         (None, 2)                 2562      
Total params: 2,260,546
Trainable params: 2,562
Non-trainable params: 2,257,984
_________________________________________________________________


In [ ]:
# fit mobilenet model
mobilenet_history = mobilenet_model.fit(train_data,
                                  epochs=EPOCHS,
                                  steps_per_epoch=len(train_data),
                                  validation_data=test_data,
                                  validation_steps=len(test_data),
                                  callbacks=[create_tensorboard_callback(dir_name="tensorflow_hub",
                                                                         experiment_name="mobilenetv2")])


Saving Tensorboard log files to : tensorflow_hub/20210626-073046
Epoch 1/5


ValueError: ignored

In [ ]:
# plot train and test loss curves
def plot_loss_curve(history):
  """
  returns separate loss curve for training and test data
  """

  loss = history.history["loss"]
  val_loss = history.history["val_loss"]

  accuracy = history.history["accuracy"]
  val_accuracy = history.history["val_accuracy"]

  epochs = range(len(history.history["loss"]))

  # plot loss
  plt.figure()
  plt.plot(epochs, loss, label="training_loss")
  plt.plot(epochs, val_loss, label="val_loss")
  plt.title("loss")
  plt.xlabel(epochs)
  plt.legend()

  # plt accuracy
  plt.figure()
  plt.plot(epochs, accuracy, label="training_accuracy")
  plt.plot(epochs, val_accuracy, label="val_accuracy")
  plt.title("accuracy")
  plt.xlabel(epochs)
  plt.legend()
  

In [ ]:
# check loss curve
plot_loss_curve(mobilenet_history)